---

author: zhangbz  
project: python-test  
date: 2022/11/2  
description:    

---

<span id="jump"><h1>目录</h1></span>
* [0-基础](#0-基础)
    * [0.1-基本使用](#0.1-基本使用)
    * [0.2-与numpy转换](#0.2-与numpy转换)


* [1-梯度与回归](#1-梯度与回归)
    * [1.1-梯度计算](#1.1-梯度计算)
        * [1.1.1-基本方法](#1.1.1-基本方法)
        * [1.1.2-计算流程](#1.1.2-计算流程)
    * [1.2-线性回归尝试](#1.2-线性回归尝试) 
        * [1.2.1-流程](#1.2.1-流程)
        * [1.2.2-使用GPU](#1.2.2-使用GPU)

In [5]:
import torch
import numpy as np

print(torch.__version__)

1.13.0+cu117


# 0.1-基本使用
[👆](#jump)

In [2]:
m1 = torch.empty(5, 3)
print(m1)

tensor([[1.6811e+03, 7.3148e-43, 1.6811e+03],
        [7.3148e-43, 1.6811e+03, 7.3148e-43],
        [1.6811e+03, 7.3148e-43, 1.6811e+03],
        [7.3148e-43, 1.6811e+03, 7.3148e-43],
        [1.6811e+03, 7.3148e-43, 1.6811e+03]])


In [3]:
m2 = torch.rand(5, 3)
print(m2)

tensor([[0.5606, 0.2570, 0.3257],
        [0.6105, 0.6487, 0.1861],
        [0.6526, 0.0969, 0.7644],
        [0.6740, 0.8087, 0.5831],
        [0.4222, 0.3694, 0.7947]])


In [4]:
m3 = torch.zeros(5, 3, dtype=torch.int)
print(m3)
print(m3.size())

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]], dtype=torch.int32)
torch.Size([5, 3])


In [5]:
print(m2[:, 1])
print(m2[2, :])

tensor([0.2570, 0.6487, 0.0969, 0.8087, 0.3694])
tensor([0.6526, 0.0969, 0.7644])


In [6]:
mm1 = torch.randn(4, 4)
mm2 = mm1.view(16)
mm3 = mm1.view(-1, 8)
print(mm2.size())
print(mm3.size())

torch.Size([16])
torch.Size([2, 8])


# 0.2-与numpy转换
[👆](#jump)

In [7]:
a = torch.ones(5)
b = a.numpy()
print(b)

[1. 1. 1. 1. 1.]


In [8]:
c = np.ones(4)
d = torch.from_numpy(c)
print(d)

tensor([1., 1., 1., 1.], dtype=torch.float64)


# 1.1-梯度计算
## 1.1.1-基本方法
[👆](#jump)

In [2]:
a = torch.randn(3, 4, requires_grad=True)
print(a)

b = torch.randn(3, 4, requires_grad=True)
t = a + b
y = t.sum()
print(y)

y.backward()
print(b.grad)

tensor([[ 0.2628,  0.8341, -0.4854, -2.3951],
        [ 0.5408,  0.1925, -0.2718, -1.2567],
        [-1.0796, -0.0208,  0.1210,  0.9444]], requires_grad=True)
tensor(-0.4718, grad_fn=<SumBackward0>)
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])


## 1.1.2-计算流程
[👆](#jump)

In [3]:
x = torch.rand(1)
b = torch.rand(1, requires_grad=True)
w = torch.rand(1, requires_grad=True)
y = w * x
z = y + b

z.backward(retain_graph=True)  # 如果梯度不清零，会累加
print(b.grad)

tensor([1.])


# 1.2-线性回归尝试
## 1.2.1-流程
[👆](#jump)

In [6]:
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1)
x_train.shape

(11, 1)

In [7]:
y_values = [2*i+i for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
y_train.shape

(11, 1)

In [8]:
import torch.nn as nn

In [9]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out

In [10]:
input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)

In [11]:
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

#### 指定参数和损失函数

In [16]:
epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
criterion = nn.MSELoss()

In [17]:
for epoch in range(epochs):
    epoch += 1

    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)

    optimizer.zero_grad()  # 清零梯度

    outputs = model(inputs)

    loss = criterion(outputs, labels)
    loss.backward()

    optimizer.step()

    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 50, loss 0.3724166750907898
epoch 100, loss 0.21241281926631927
epoch 150, loss 0.12115223705768585
epoch 200, loss 0.06910073012113571
epoch 250, loss 0.039412423968315125
epoch 300, loss 0.022479349747300148
epoch 350, loss 0.012821429409086704
epoch 400, loss 0.007312828674912453
epoch 450, loss 0.0041710068471729755
epoch 500, loss 0.00237897876650095
epoch 550, loss 0.0013568735448643565
epoch 600, loss 0.0007739158463664353
epoch 650, loss 0.0004414145660120994
epoch 700, loss 0.00025176239432767034
epoch 750, loss 0.00014359611668623984
epoch 800, loss 8.189976506400853e-05
epoch 850, loss 4.671609349315986e-05
epoch 900, loss 2.6644685931387357e-05
epoch 950, loss 1.5197255379462149e-05
epoch 1000, loss 8.668936061440036e-06


#### 测试

In [18]:
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

array([[5.47664892e-03],
       [3.00468802e+00],
       [6.00389910e+00],
       [9.00311089e+00],
       [1.20023222e+01],
       [1.50015335e+01],
       [1.80007439e+01],
       [2.09999561e+01],
       [2.39991665e+01],
       [2.69983768e+01],
       [2.99975891e+01]], dtype=float32)

#### 保存/读取模型

In [ ]:
torch.save(model.state_dict(),'model.pkl')

In [ ]:
model.load_state_dict(torch.load('model.pkl'))

## 1.2.2-使用GPU
[👆](#jump)

In [ ]:
import torch
import numpy as np
import torch.nn as nn

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out
    
input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)

# 指定设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
criterion = nn.MSELoss()

for epoch in range(epochs):
    epoch += 1

    inputs = torch.from_numpy(x_train).to(device)
    labels = torch.from_numpy(y_train).to(device)

    optimizer.zero_grad()  # 清零梯度

    outputs = model(inputs)

    loss = criterion(outputs, labels)
    loss.backward()

    optimizer.step()

    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))